<a href="https://colab.research.google.com/github/mojtabaSefidi/Event-Detection-from-Twitter/blob/main/Main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prerequisites

In [ ]:
!git clone https://github.com/mojtabaSefidi/EventDetection.git/

Cloning into 'EventDetection'...
remote: Enumerating objects: 1180, done.
remote: Counting objects: 100% (312/312), done.
remote: Compressing objects: 100% (150/150), done.
remote: Total 1180 (delta 178), reused 282 (delta 155), pack-reused 868
Receiving objects: 100% (1180/1180), 40.90 MiB | 26.81 MiB/s, done.
Resolving deltas: 100% (698/698), done.


In [ ]:
%cd EventDetection

/content/EventDetection


In [ ]:
!pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 458.9/458.9 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 15.2 MB/s eta 0:00:00
  Preparing metad

## Dataset


In this section you should specify the dataset in which you want to detect the events. Your proposed dataset should at least has these two columns. if you don't have your own dataset, you can download ours.

1) text

2) datetime

3) hashtag (opitonal)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
from utils import read_dataset, get_hdbscan

final_dataset = read_dataset(dataset_path = '/content/gdrive/MyDrive/Event Detection from Twitter/Event Detection Final Dataset/20Feb_20March_Processed.csv', # path to dataset
                             datetime_column = 'datetime', # name of datetime column
                             hashtag_column= 'hashtags' # name of hashtag column
                             )

clustering_model = get_hdbscan() # hdbscan clustering model (you can use any other clustering models)

## Trend Detection

For this section, you need a trained model for trend detection, an embedding matrix for embedding layer, a tokenizer and vocab size. You can also train the trend detection in your labeled dataset or even implement your proposed trend detection model and use it in the trending data extraction component. Note that it's advisable to use our proposed architecture and train the model in your own dataset.

In [ ]:
import shutil

!gdown -q 1-7ueGM5CFh1R3oHhMctDStqrib821K4L
!gdown -q 1-9AyraMu1oZKWsw6HDWQmWNmE3nu-dVK
!gdown -q 1-RUNaJioBCAR204RWBagoD5rqvKJo3nT
!gdown -q 1pKIxul6UADQyx7TaaCAkUkw6OSXQDWiu

!unzip -q /content/EventDetection/Model_Checkpoints.zip
!rm '/content/EventDetection/Model_Checkpoints.zip'

shutil.move("/content/EventDetection/content/gdrive/MyDrive/Model Checkpoints", "/content/EventDetection")
shutil.rmtree('/content/EventDetection/content')

In [ ]:
import pickle
with open('text2sequence_tokenizer.pkl','rb') as f:
  text2sequence_tokenizer = pickle.load(f)
vocab_size=len(text2sequence_tokenizer.word_index)+1

with open('embedding_matrix_trend_detection.pkl','rb') as f:
  embedding_matrix = pickle.load(f)

with open('X_test_trend_detection.pkl','rb') as f:
  X_test_sequence = pickle.load(f)
  max_length = X_test_sequence.shape[1]

model_checkpoint_path = 'Model Checkpoints/TrendDetection.ckpt'

In [ ]:
from Trend_Detection.predict import Trending_Data_Extractor
trending_data_extractor = Trending_Data_Extractor(embedding_matrix, # embedding matrix
                                                  model_checkpoint_path, # trend detection model
                                                  text2sequence_tokenizer, # tokenizer
                                                  max_length # maximum path
                                                  )

## Main pipeline

In [ ]:
from Event_Detection_Pipeline.entity_clustering import Entity_Clustering
from Event_Detection_Pipeline.entity_extraction import Extract_Entities
from Event_Detection_Pipeline.event_detection_pipeline import Event_Detection_Pipeline

In [ ]:
%%time
main = Event_Detection_Pipeline(
    object_entity_extraction = Extract_Entities(dataset=final_dataset, #dataset
                                               embedding_model_path = 'HooshvareLab/bert-fa-zwnj-base', # word embedding model path
                                               NER_model_path = 'HooshvareLab/bert-base-parsbert-ner-uncased', # NER model path
                                               trending_data_extractor = None # trending data extractor object (not necessary)
                                               ),

    object_entity_clustering=Entity_Clustering(clustering_model), # clustering model
    start_date='2023-02-28', # start date
    end_date='2023-03-14', # end date
    result_storage_path='/content/Results', # path for storing the results
    window_size='1D', # window size, for ex : 1D/ 2D/ 1W/ ...
    window_length=500, # number of tweet in each window
    n_common_entity_threshold=3, # minimum number of entity to link the nodes
    # clusters_dict_saving_path = 'AllClusters.pkl', #path for saving the clusters
    segmented_dfs_directory='/content/gdrive/MyDrive/Event Detection from Twitter/Final Results/Detail_EnrichEvent3/Segmented_Dfs', # path of massage block (if available)
    # segmented_dfs_saving_path = 'Segmented_Dfs', # path directory to store the segmented data-frames
    name_entity_loading_path = '/content/gdrive/MyDrive/Event Detection from Twitter/Final Results/Detail_EnrichEvent3/AllNameEntities.pkl', # path of detected entities (if available)
    # NE_occurrence_dict_saving_path = None, # path for saving the detected entities
    filter_trends=False, # turn on/off trending data extraction component
    trending_threshold=0.1, # minimum thresholf for trending data extraction component
    visualize=True # visualize feature space or not
    )


Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


All model checkpoint layers were used when initializing TFBertForTokenClassification.

All the layers of TFBertForTokenClassification were initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-ner-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForTokenClassification for predictions without further training.


---------------------------------------- Working on window 1 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. Calculate embedding matrix...
4. Cluster name entities...
==== -1 / 94
---------------------------------------- Elapsed Time : 00:00:09
---------------------------------------- Working on window 2 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. Calculate embedding matrix...
4. Cluster name entities...
==== -1 / 75
---------------------------------------- Elapsed Time : 00:00:07
---------------------------------------- Working on window 3 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. Calculate embedding matrix...
4. Cluster name entities...
==== -1 / 87
---------------------------------------- Elapsed Time : 00:00:07
---------------------------------------- Working on window 4 with 500 samples
1. Restoring name entities...
2. Calculate co-occurrence matrix...
3. 

## Generating final output

In [ ]:
from Event_Summarization.predict import Summary_Generator
from Output_Generator.output_generator import Output_Generator

In [ ]:
output_generator = Output_Generator(summarization_model_path='/content/gdrive/MyDrive/Event Detection from Twitter/Models and variables/Summerization2.bin', # checkpoint of summarization moedl
                                    gpt_model_path = 'flax-community/gpt2-medium-persian', # path of language model
                                    embedding_model_path='HooshvareLab/bert-fa-zwnj-base' # path of embedding model
                                    )

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
output_generator.export(results_path='/content/Results') # path of results directory

-------------------------- Working on event 1 --------------------------
1) Drawing WordCloud...
2) Generating Summary...
   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...
3) Extract Events' Attributs...
-------------------------- Working on event 10 --------------------------
1) Drawing WordCloud...
2) Generating Summary...
   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...
3) Extract Events' Attributs...
-------------------------- Working on event 2 --------------------------
1) Drawing WordCloud...
2) Generating Summary...
   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...
3) Extract Events' Attributs...
---------------

## Event Summarization

You can also fine-tune your own summarization model using the scripts in synthetic_dataset.py and train.py

In [ ]:
summary_genarator = Summary_Generator(summarization_model_path='/content/gdrive/MyDrive/Event Detection from Twitter/Models and variables/Summerization2.bin', # summarization model checkpoint
                                      gpt_model_path = 'flax-community/gpt2-medium-persian', # language model path
                                      embedding_model_path='HooshvareLab/bert-fa-zwnj-base' # embedding model path
                                      )

Some weights of BertModel were not initialized from the model checkpoint at HooshvareLab/bert-fa-zwnj-base and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
dict_summary, list_summary = summary_genarator.generate_event_summary(event_path='/content/Results/event_6') # path to event directory
dict_summary

   2.1) Retrieve key phrases...
   2.2) Produce clusters...
   2.3) Filter clusters...
   2.4) Generate summerizations...
   2.5) Filter summaries and return the best one...


{2: ([0.542, 0.7475, 0.6544, 0.6227, 0.696, 0.7161, 0.7441, 0.6733], 0.67451),
 3: ([0.5726, 0.7143, 0.6588, 0.6844, 0.7033, 0.5951, 0.7409, 0.712], 0.67268),
 4: ([0.58, 0.6603, 0.6046, 0.5943, 0.7441, 0.7102, 0.7008, 0.6313], 0.6532),
 1: ([0.455, 0.4876, 0.5146, 0.4974, 0.5456, 0.6927, 0.5859, 0.464], 0.53035),
 0: ([0.48, 0.4871, 0.4211, 0.4396, 0.501, 0.6534, 0.5247, 0.4564], 0.49541)}